In [25]:
import pandas as pd
from siuba import *
import shared_utils

from calitp_data_analysis import get_fs
from segment_speed_utils import helpers, time_series_utils
from segment_speed_utils.project_vars import SCHED_GCS, SEGMENT_GCS, GTFS_DATA_DICT, analysis_date

i'd start with 5, checking if the columns you want are already there in this file (saved out at the end of stop_arrivals_to_speed, and is a speeds by trip table for that day)

In [28]:
catalog = shared_utils.catalog_utils.get_catalog('gtfs_analytics_data')

In [29]:
catalog.speedmap_segments

{'dir': '${gcs_paths.SEGMENT_GCS}', 'stage1': '${speeds_tables.vp_dwell}', 'proxy_stop_times': 'stop_time_expansion/speedmap_stop_times', 'stage2': 'nearest/nearest_vp_speedmap_proxy', 'stage2b': 'nearest/nearest2_vp_speedmap_proxy', 'stage3': 'speedmap/stop_arrivals_proxy', 'stage3b': 'speedmap/stop_arrivals', 'stage4': 'speedmap/speeds', 'trip_stop_cols': ['trip_instance_key', 'stop_sequence', 'stop_sequence1'], 'shape_stop_cols': ['shape_array_key', 'shape_id', 'route_id', 'route_short_name'], 'stop_pair_cols': ['stop_pair', 'stop_pair_name', 'segment_id'], 'segments_file': 'segment_options/speedmap_segments', 'shape_stop_single_segment': 'rollup_singleday/speeds_shape_speedmap_segments', 'route_dir_single_segment': 'rollup_singleday/speeds_route_dir_speedmap_segments', 'route_dir_multi_segment': 'rollup_multiday/speeds_route_dir_speedmap_segments', 'min_trip_minutes': '${speed_vars.time_min_cutoff}', 'max_trip_minutes': 180, 'max_speed': '${speed_vars.max_speed}', 'min_meters_elaps

In [30]:
analysis_date

'2024-08-14'

In [31]:
path = f'{catalog.speedmap_segments.dir}{catalog.speedmap_segments.stage4}_{analysis_date}.parquet'

In [32]:
path

'gs://calitp-analytics-data/data-analyses/rt_segment_speeds/speedmap/speeds_2024-08-14.parquet'

In [33]:
df = pd.read_parquet(path)

In [34]:
df.columns

Index(['trip_instance_key', 'stop_sequence', 'stop_sequence1',
       'shape_array_key', 'stop_meters', 'arrival_time', 'arrival_time_sec',
       'subseq_arrival_time_sec', 'subseq_stop_meters', 'meters_elapsed',
       'sec_elapsed', 'speed_mph', 'schedule_gtfs_dataset_key', 'route_id',
       'direction_id', 'route_short_name', 'scheduled_service_minutes',
       'sched_rt_category', 'time_of_day', 'shape_id', 'segment_id',
       'stop_pair_name', 'stop_pair'],
      dtype='object')

In [36]:
route_info = helpers.import_scheduled_trips(
    analysis_date,
    columns=[
        "gtfs_dataset_key", "trip_instance_key",
        "route_id", "direction_id", "route_short_name"
    ],
    get_pandas=True,
)

In [37]:
route_info

,schedule_gtfs_dataset_key,trip_instance_key,route_id,direction_id,route_short_name
0,1770249a5a2e770ca90628434d4934b1,7406077cb3ef2783f260ac5ddf06f884,3407,1.0,Route 18
1,1770249a5a2e770ca90628434d4934b1,d9f1f0a4d53997858a48a340772c75a3,3408,0.0,Route 21
2,1770249a5a2e770ca90628434d4934b1,3c36b0976b9cd77a0862224d842fea5f,3408,0.0,Route 21
3,1770249a5a2e770ca90628434d4934b1,a26bcf0a35ac9342d82435bb064f8700,3408,0.0,Route 21
4,1770249a5a2e770ca90628434d4934b1,77b979dca36b0f4fd305210cee23f6a1,3408,0.0,Route 21
...,...,...,...,...,...
98814,c4092405159366c705b62df938293a4e,9f5b9be0653ea22d7c705fb8ccecf20f,Orange County Line,0.0,None
98815,c4092405159366c705b62df938293a4e,c9284435d85c3cf9a5fa5afacbcfb5f0,Antelope Valley Line,1.0,None
98816,c4092405159366c705b62df938293a4e,23bdf6228677587ad66658b818f8f3c8,Orange County Line,1.0,None
98817,c4092405159366c705b62df938293a4e,633996f6d703cbff7838099bc4e8c6e0,San Bernardino Line,0.0,None


In [49]:
test = ['x', 'y', 'x']

In [50]:
list(dict.fromkeys(test))

['x', 'y']